In [1]:
!git clone https://@github.com/Web-Team-IITI-Gymkhana/data-bot.git

Cloning into 'data-bot'...
fatal: could not read Password for 'https://github.com': No such device or address


# Basic Libraries Import

---



In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir(os.curdir))
from sklearn import preprocessing
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier

['.config', 'predicted_company_scores.csv', 'GoodCom.csv', 'sample_data']


# Data Import

---

Companies and their CIK

In [3]:
com= pd.read_csv('/content/GoodCom.csv')

com.head(3)

,CIK,CompanyName
0,1459417,2U INC
1,1023731,8X8 INC
2,1580808,A10 NETWORKS INC




---

#Analysis: Finding Threshold To Classify Investability as Good / Neutral / Bad 

>> Regression Analysis Results

In [4]:
predscore= pd.read_csv('/content/predicted_company_scores.csv')
predscore.head()

,cik_date,GrossProfit,GrossMargin,WorkingCapitalRatio,EarningPerShare,DebtToEquityRatio,PEratio,ReturnOfEquity,EBIDTAratio,EvRatio,...,growth_rate_label,profitm_label,grossm_label,ro40_label,churnrate_label,EVbyEbidta_label,marketCap_label,magicNum_label,aggregate,predicted_aggregate
0,1459417_2019,865736136.26,0.71,1.67,-0.34,0.13,-70.75,-1.80,209348403.18,42770580833.64,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.15,0.15
1,1459417_2020,865736136.26,0.71,2.55,-2.99,0.14,-13.39,-13.15,209348403.18,42770580833.64,...,1.00,1.00,0.00,1.00,0.00,0.00,1.00,0.00,0.31,0.42
2,1459417_2021,865736136.26,0.71,1.18,-2.57,0.19,-7.81,-11.22,209348403.18,42770580833.64,...,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.08,0.08
3,1023731_2019,245394000.00,0.70,5.32,0.12,0.15,165.58,2.31,-90988000.00,42770580833.64,...,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.23,0.23
4,1023731_2020,301224000.00,0.68,2.25,-1.67,0.20,-8.30,-27.56,-159819000.00,42770580833.64,...,0.00,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.15,0.16


>>Manually Finding Accuracy of Regression Analysis using XGBRegressor

In [5]:
diff=abs(predscore['predicted_aggregate']-predscore['aggregate'])
count=0
for x in diff:
  if x<=0.1 and x>=0:
    count+=1
print((count/len(diff))*100)    

97.91666666666666


>>Function to Extract CIK from CIK_Year Format

In [6]:
def cik_scrap(st):
  st=str(st)
  return float(st.split('_')[0])

In [7]:
predscore['CIK']= predscore['cik_date'].apply(cik_scrap)
predscore.head()

,cik_date,GrossProfit,GrossMargin,WorkingCapitalRatio,EarningPerShare,DebtToEquityRatio,PEratio,ReturnOfEquity,EBIDTAratio,EvRatio,...,profitm_label,grossm_label,ro40_label,churnrate_label,EVbyEbidta_label,marketCap_label,magicNum_label,aggregate,predicted_aggregate,CIK
0,1459417_2019,865736136.26,0.71,1.67,-0.34,0.13,-70.75,-1.80,209348403.18,42770580833.64,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.15,0.15,1459417.00
1,1459417_2020,865736136.26,0.71,2.55,-2.99,0.14,-13.39,-13.15,209348403.18,42770580833.64,...,1.00,0.00,1.00,0.00,0.00,1.00,0.00,0.31,0.42,1459417.00
2,1459417_2021,865736136.26,0.71,1.18,-2.57,0.19,-7.81,-11.22,209348403.18,42770580833.64,...,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.08,0.08,1459417.00
3,1023731_2019,245394000.00,0.70,5.32,0.12,0.15,165.58,2.31,-90988000.00,42770580833.64,...,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.23,0.23,1023731.00
4,1023731_2020,301224000.00,0.68,2.25,-1.67,0.20,-8.30,-27.56,-159819000.00,42770580833.64,...,0.00,1.00,0.00,1.00,0.00,0.00,0.00,0.15,0.16,1023731.00


>>**Creating DataFrame 'CIK'  'aggregate' & 'predicted_aggregate' features**

In [8]:
fans= predscore[['CIK','aggregate','predicted_aggregate']]
fans.head()

,CIK,aggregate,predicted_aggregate
0,1459417.00,0.15,0.15
1,1459417.00,0.31,0.42
2,1459417.00,0.08,0.08
3,1023731.00,0.23,0.23
4,1023731.00,0.15,0.16


>> **Merging Companies List with Test Data**

In [9]:
final=pd.merge(fans, com, on='CIK')
final=pd.DataFrame(final.to_numpy(),columns=final.keys())
final.head()

,CIK,aggregate,predicted_aggregate,CompanyName
0,1459417.00,0.15,0.15,2U INC
1,1459417.00,0.31,0.42,2U INC
2,1459417.00,0.08,0.08,2U INC
3,1023731.00,0.23,0.23,8X8 INC
4,1023731.00,0.15,0.16,8X8 INC


>> Mapping CIK number with Company Name

In [10]:
cik_com_map=dict(zip(final.CIK,final.CompanyName))
#cik_com_map

>> **Grouping Companies to Get Average Performance **

In [11]:
final2=final.groupby(['CompanyName']).mean()
final2.head()

,CIK,aggregate,predicted_aggregate
CompanyName,,,
2U INC,1459417.00,0.18,0.22
8X8 INC,1023731.00,0.21,0.20
A10 NETWORKS INC,1580808.00,0.42,0.42
ACI WORLDWIDE INC,935036.00,0.44,0.39
ADOBE INC,796343.00,0.51,0.51


>>*Scores Achieved by companies out of 100*

In [12]:
def multiplier(n):
  return n*100

final2['aggregate']=  final2['aggregate'].apply(multiplier)
final2['predicted_aggregate']= final2['predicted_aggregate'].apply(multiplier)

In [13]:
final2.head()

,CIK,aggregate,predicted_aggregate
CompanyName,,,
2U INC,1459417.00,17.95,21.67
8X8 INC,1023731.00,20.51,20.14
A10 NETWORKS INC,1580808.00,42.31,42.44
ACI WORLDWIDE INC,935036.00,43.59,38.75
ADOBE INC,796343.00,51.28,51.32


# Statistical Distribution of Score to Calculate Threshold

In [14]:
final2.describe()

,CIK,aggregate,predicted_aggregate
count,160.00,160.00,160.00
mean,1262922.05,27.04,26.95
std,344890.77,13.16,12.93
min,8670.00,0.00,0.02
25%,1022237.50,17.95,17.97
50%,1351460.50,25.64,25.25
75%,1521904.50,34.62,34.03
max,1810806.00,61.54,62.90


# Classification of Companies 

---



In [15]:
def bng(n):
  if n<=15:
    return 'Bad'
  elif n>15 and n<=35:
    return 'Neutral'
  else: 
    return 'Good'

final2['Investability']= final2['aggregate'].apply(bng)

In [16]:
final2.head(10)

,CIK,aggregate,predicted_aggregate,Investability
CompanyName,,,,
2U INC,1459417.00,17.95,21.67,Neutral
8X8 INC,1023731.00,20.51,20.14,Neutral
A10 NETWORKS INC,1580808.00,42.31,42.44,Good
ACI WORLDWIDE INC,935036.00,43.59,38.75,Good
ADOBE INC,796343.00,51.28,51.32,Good
AGILYSYS INC,78749.00,28.21,30.98,Neutral
ALARM.COM HOLDINGS INC,1459200.00,46.15,46.15,Good
ALTAIR ENGINEERING INC,1701732.00,28.85,30.29,Neutral
ALTERYX INC,1689923.00,35.90,34.57,Good


In [17]:
final2.to_csv('Investability.csv')